<a href="https://colab.research.google.com/github/TuragDev/ML-Python-Codes/blob/main/crystal_system_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-optimize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 3.1 MB/s eta 0:00:00


In [32]:
!pip install xgboost

In [ ]:
from google.colab import files
uploaded= files.upload()

Saving Orbital_mp_test.csv to Orbital_mp_test.csv
Saving orbital_MP_train.csv to orbital_MP_train.csv
Saving sine_mp_test.csv to sine_mp_test.csv
Saving Sine_MP_train.csv to Sine_MP_train.csv
Saving Xrd_MP_train.csv to Xrd_MP_train.csv
Saving xrd_test_mp.csv to xrd_test_mp.csv


In [ ]:
import joblib
from sklearn.model_selection import train_test_split
import numpy as np

import pandas as pd

from sklearn.model_selection import GridSearchCV, RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.model_selection import train_test_split

import numpy as np

import pickle
import pandas as pd
from sklearn.preprocessing import LabelEncoder

import sys
# In[3]:


from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold
from skopt import BayesSearchCV


In [ ]:
df1= pd.read_csv('Xrd_MP_train.csv')

df2= pd.read_csv('xrd_test_mp.csv')
df=pd.concat([df1,df2],ignore_index=True)

In [ ]:
X=df.iloc[:,14:]
Y=df['crystal_system']

In [ ]:
from sklearn.preprocessing import StandardScaler
# create a scaler object
scaler = StandardScaler()

# fit scaler on the training data
scaler.fit(X)

# apply the scaler to both the training and test data
X_normalized = scaler.transform(X)

In [ ]:
label_encoder = LabelEncoder()

# Fit and transform label encoder on target values
y_encoded = label_encoder.fit_transform(Y)

In [35]:
import xgboost
from hyperopt import fmin, tpe, hp

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y_encoded, test_size=0.2, random_state=108)

In [48]:
# Define the search space for hyperparameters
param_space = {
    'max_depth': hp.quniform('max_depth', 3, 10, 1),
    'gamma': hp.uniform('gamma', 0, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0, 5),
    'reg_lambda': hp.uniform('reg_lambda', 0, 5),
    'learning_rate': hp.loguniform('learning_rate', 0.01, 3),
    'n_estimators': hp.quniform('n_estimators', 10, 500, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0),
    'subsample': hp.uniform('subsample', 0.6, 1.0),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1)
}

# Define the objective function to minimize (negative accuracy)
def objective(params):
    model = XGBClassifier(
        max_depth=int(params['max_depth']),
        gamma=params['gamma'],
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda'],
        learning_rate=params['learning_rate'],
        n_estimators=int(params['n_estimators']),
        min_child_weight=int(params['min_child_weight'])
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return -accuracy  # Minimize negative accuracy

# Perform Bayesian optimization
best = fmin(
    fn=objective,
    space=param_space,
    algo=tpe.suggest,
    max_evals=50  # Number of optimization iterations
)

print("Best Hyperparameters:", best)

100%|██████████| 50/50 [03:31<00:00,  4.23s/trial, best loss: -0.6778523489932886]
Best Hyperparameters: {'colsample_bytree': 0.7949731238966005, 'gamma': 0.48856866454029474, 'learning_rate': 1.7830907150782347, 'max_depth': 6.0, 'min_child_weight': 5.0, 'n_estimators': 449.0, 'reg_alpha': 2.42245255243136, 'reg_lambda': 3.09328824076608, 'subsample': 0.6869716695502667}


In [49]:
import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import KFold, cross_val_score

# Define the best hyperparameters
best_hyperparameters = {
    'n_estimators': 449,
    'learning_rate': 1.7830907150782347,
    'gamma': 0.48856866454029474,
    'max_depth': 6,
    'min_child_weight': 5,
    'colsample_bytree': 0.7949731238966005,
    'subsample': 0.6869716695502667,
    'reg_alpha': 2.42245255243136,
    'reg_lambda': 3.09328824076608
}

# Create an XGBoost classifier with the best hyperparameters
model = XGBClassifier(**best_hyperparameters)

# Perform k-fold cross-validation
cv = RepeatedKFold(n_splits=5, n_repeats=10, random_state=1)
scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
mean_accuracy = scores.mean()
print("Mean Accuracy:", mean_accuracy)

Mean Accuracy: 0.6811974789915967


In [50]:
model.fit(X_train, y_train)
pred = model.predict(X_train)
training_accuracy= accuracy_score(y_train,pred)
print("Accuracy on Training data: ",training_accuracy)
pred = model.predict(X_test)
test_accuracy= accuracy_score(y_test,pred)
print("Accuracy on Test data: ",test_accuracy)

Accuracy on Training data:  0.9681208053691275
Accuracy on Test data:  0.6375838926174496
